# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Нужно довести долю правильных ответов *accuracy* по крайней мере до 0.75.

## Откроем и изучим файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Разобьем данные на выборки

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

print(f'Исходные данные разбиты на обучающую, валидационную и тестовую выборку в соотношении {len(df_train)/len(df):.0%}, {len(df_valid)/len(df):.0%}, {len(df_test)/len(df):.0%} соответственно.')

Исходные данные разбиты на обучающую, валидационную и тестовую выборку в соотношении 60%, 20%, 20% соответственно.


Выделим для каждой выборки данные признаков (features: число звонков, минуты разговора, число сообщений, использование интернет-трафика) и целевых признаков (target: тариф ультра или нет).

In [6]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

## Исследуем модели

Ипортируем библиотеки для создания моделей дерева решений, случайного леса и линейной регрессии. Также создадим списки для добавления в них метрик оценки моделей.

In [7]:
accuracy_valid = []
accuracy_test = []
time = []

**Модель дерерво решений (DT)** настроим по гиперпараметру глубины дерева в диапазоне от 1 до 10 и сохраним модель с наибольшим accuracy.

In [8]:
best_model_DT = None
best_result_DT = 0
best_depth_DT = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # инициализация модели с заданной глубиной дерева
    model.fit(features_train, target_train) # обучаем модель
    predictions_valid = model.predict(features_valid) # получаем предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # считаем качество модели
    if result > best_result_DT:  #проверяем модель на метрику accuracy
        best_model_DT = model
        best_result_DT = result
        best_depth_DT = depth
        
accuracy_valid.append(best_result_DT)        
print(f"Accuracy лучшей модели дерева решений на валидационной выборке: {best_result_DT:.4f}, при глубине дерева {best_depth_DT}.")

Accuracy лучшей модели дерева решений на валидационной выборке: 0.7854, при глубине дерева 3.


**Модель случайный лес (RF)** настроим по гиперпараметрам числа деревьев от 1 до 50 и глубины дерева от 1 до 10. Сохраним модель с наибольшим accuracy.

In [9]:
best_model_RF = None
best_result_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in range(1, 51):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result_RF:
            best_model_RF = model
            best_result_RF = result
            best_est_RF = est
            best_depth_RF = depth

accuracy_valid.append(best_result_RF)            
print(f"Accuracy лучшей модели случайного леса на валидационной выборке: {best_result_RF:.4f}, при количестве деревьев {best_est_RF} и глубине дерева {best_depth_RF}.")

Accuracy лучшей модели случайного леса на валидационной выборке: 0.8087, при количестве деревьев 40 и глубине дерева 8.


**Модель логистическая регрессия (LR)** настроим по гиперпараметру "С" (силы регуляризации) от 0,01 до 5. Сохраним модель с наболшим accuracy.

In [10]:
best_model_LR = None
best_result_LR = 0
best_c_LR = 0
for c in np.arange(0.01, 5.01, 0.01):
    model = LogisticRegression(random_state=12345, C=c, solver='liblinear') 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result_LR:
            best_model_LR = model
            best_result_LR = result 
            best_c_LR = c
            
accuracy_valid.append(best_result_LR)
print(f"Accuracy лучшей модели логистической регрессии на валидационной выборке: {best_result_LR:.4f}, при гиперпараметре 'с' равном {best_c_LR:.4f}")

Accuracy лучшей модели логистической регрессии на валидационной выборке: 0.7589, при гиперпараметре 'с' равном 0.8200


## Проверим модели на тестовой выборке

In [11]:
models =(best_model_DT, best_model_RF, best_model_LR)
models_name = ['дерево решений','случайный лес','логистическая регрессия']
models_d = dict(zip(models, models_name))
for model in models_d:
    predictions_test = model.predict(features_test)
    result = accuracy_score(target_test, predictions_test)
    accuracy_test.append(result)
    print(model)
    print(f"Accuracy модели {models_d.get(model)} на тестовой выборке {result}", end='\n\n')


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
Accuracy модели дерево решений на тестовой выборке 0.7791601866251944

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
Accuracy модели случайный лес на тесто

Создади функцию предсказания целевого признака на основе признаков тестовой выборки данных для проведения теста на скорость и получения TimeitResult при 100 повторениях. 

In [12]:
def f(model):
    return model.predict(features_test)

for model in models_d:
    t = %timeit -n100 -o f(model)
    time.append(t)

807 µs ± 46.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.66 ms ± 75.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
724 µs ± 32.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Выделим из полученных данных TimeitResult необходимые параметры.

In [13]:
average_time = []
stdev_time = []
for i in range(len(time)):
    average_time.append(time[i].average)
    stdev_time.append(time[i].stdev)

Сведем полученные метрики по каждой модели в таблицу.

In [14]:
columns = ['accuracy_valid','accuracy_test','среднее время предсказания', 'ср. квадратичное отклонение времени предсказания']
result_data = pd.DataFrame(data=(accuracy_valid, accuracy_test, average_time, stdev_time), columns=models_name, index=columns)
result_data

,дерево решений,случайный лес,логистическая регрессия
accuracy_valid,0.785381,0.808709,0.758942
accuracy_test,0.779160,0.796267,0.740280
среднее время предсказания,0.000807,0.008665,0.000724
ср. квадратичное отклонение времени предсказания,0.000047,0.000075,0.000033


## Проверим модели на адекватность

Будем считать, что если accuracy модели выше, чем у предположенных вариантов предсказания, то модель проходит проверку на адекватность.

Для проверки модели на адекватность посчитаем доли клиентов каждого тарифа. 

In [15]:
smart = (df['is_ultra']==0).sum() / df.shape[0]
ultra = 1-smart
print("Доля клинтов с тарифом Smart", smart)
print("Доля клинтов с тарифом Ultra", ultra)

Доля клинтов с тарифом Smart 0.693528313627878
Доля клинтов с тарифом Ultra 0.30647168637212197


Выдвенем три предположения:  
1) Будем утверждать, что все пользователи должны пользоваться тарифом с наибольшей долей (Smart);  
2) Семи из десяти клиентов будем предлагать тариф Smart, а трем из десяти тариф Ultra;  
3) Для каждого случая будем предсказывать тариф случайным образом.

In [16]:
accuracy_only_smart = 1*smart
accuracy_proportional = smart*smart+ultra*ultra
accuracy_random = 0.5*0.5+0.5*0.5
print("Вероятность правильного ответа по предположению 1)", accuracy_only_smart)
print("Вероятность правильного ответа по предположению 2)", accuracy_proportional)
print("Вероятность правильного ответа по предположению 3)", accuracy_random)

Вероятность правильного ответа по предположению 1) 0.693528313627878
Вероятность правильного ответа по предположению 2) 0.5749064163513007
Вероятность правильного ответа по предположению 3) 0.5


## Вывод 
По предоставленным данным обученым и валидированы три модели: дерево решений, случайный лес и логистическая регрессия.  
Все модели на тестовой выборке имеют accuracy больше 0,75.  
Модель случайный лес на валидационных данных имеет наибольший accuracy 0,8087, однако другие модели имеют accuracy не ниже 0,75, и при этом имеют среднее время предсказания более чем в 10 раз ниже.  
Оптимальным по соотношению accuracy и длительности предсказания является модель дерева решения, устапая в точности модели случайного леса на 2,5%, но предсказывая в 10 раз быстрее.